In [2]:
import os
import pandas as pd

In [3]:
df = pd.read_csv('/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv')

In [4]:
df

,user_id,book_id,rating,decade,original_title,authors,genres
0,1,258,5,2000,La sombra del viento,"Carlos Ruiz Zafón, Lucia Graves","Mystery, Historical"
1,2,4081,4,2000,NaN,NaN,NaN
2,2,260,5,1930,How to Win Friends and Influence People,Dale Carnegie,"Nonfiction, Drama"
3,2,9296,5,1970,Das Drama des begabten Kindes und die Suche na...,"Alice Miller, Ruth Ward","Horror, Mystery"
4,2,2318,3,1990,The Millionaire Next Door: The Surprising Secr...,"Thomas J. Stanley, William D. Danko","Nonfiction, Drama"
...,...,...,...,...,...,...,...
5976474,49925,510,5,1990,The Great Hunt,Robert Jordan,"Fantasy, Adventure"
5976475,49925,528,4,1990,The Dragon Reborn,Robert Jordan,"Classics, Drama"
5976476,49925,722,4,1990,The Shadow Rising,Robert Jordan,"Adventure, Drama"
5976477,49925,949,5,1990,The Fires of Heaven,Robert Jordan,"Fantasy, Adventure"


## data analysis

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# --- Your canonical 13 genres, fixed order for rows/cols ---
GENRES_13 = [
    "Adult", "Adventure", "Children's", "Classics", "Drama",
    "Fantasy", "Historical", "Horror", "Mystery", "Nonfiction",
    "Romance", "Science Fiction", "Thriller"
]

def parse_genres(s):
    if pd.isna(s):
        return []
    return [g.strip() for g in str(s).split(",") if g.strip()]

def to_canonical_set(gstr):
    glist = [g for g in parse_genres(gstr) if g in GENRES_13]
    # de-duplicate while preserving first-seen order
    return list(dict.fromkeys(glist))

def analyze_ordered_cooccurrence(df):
    """
    Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    """
    print("=== ORDERED CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books and keep one ordered genre list per book
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # Parse + filter to canonical 13, preserving order within each book
    books['genre_list'] = books['genres'].apply(parse_genres).apply(
        lambda gl: [g for g in gl if g in GENRES_13]
    )
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 2) Count and print how many UNIQUE canonical genres are actually present in df
    present = []
    for gl in books['genre_list']:
        present.extend(gl)
    present = sorted(set(present), key=lambda g: GENRES_13.index(g))
    
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 3) Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        # First occurrence positions per genre in this book (preserve order)
        pos = {}
        for k, g in enumerate(gl):
            if g not in pos:
                pos[g] = k
        
        # Count ordered pairs where Gi occurs BEFORE Gj (Gi != Gj). No diagonal.
        genres_in_book = list(pos.keys())
        for gi in genres_in_book:
            for gj in genres_in_book:
                if gi == gj:
                    continue
                if pos[gi] < pos[gj]:
                    co_mat[idx[gi], idx[gj]] += 1
    
    # Wrap into a 13x13 DataFrame with fixed order (diagonal is guaranteed 0)
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Ordered Co-occurrence Matrix:")
    print(co_df)
    return co_df

def analyze_symmetric_cooccurrence(df):
    """
    Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    """
    print("\n=== SYMMETRIC CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books; keep one genres string per book_id
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # 2) Parse & filter to canonical 13; also de-duplicate within a book
    books['genre_list'] = books['genres'].apply(to_canonical_set)
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 3) Report how many of the canonical 13 actually appear
    present = sorted(set(g for gl in books['genre_list'] for g in gl), key=lambda g: GENRES_13.index(g))
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 4) Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    #    Cell (Gi, Gj) = number of UNIQUE books that contain BOTH Gi and Gj (order-agnostic)
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        s = list(set(gl))           # ensure uniqueness before making pairs
        for g1, g2 in combinations(s, 2):
            i, j = idx[g1], idx[g2]
            # increment both directions to enforce symmetry
            co_mat[i, j] += 1
            co_mat[j, i] += 1
    
    # Zero the diagonal (no GiGi counts)
    np.fill_diagonal(co_mat, 0)
    
    # 5) Wrap in DataFrame
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Symmetric Co-occurrence Matrix:")
    print(co_df)
    return co_df

def main_analysis(df):
    """
    Run both analyses on the provided dataframe
    """
    print("Starting Genre Co-occurrence Analysis")
    print(f"Total rows in dataset: {len(df)}")
    
    # Run ordered analysis
    ordered_matrix = analyze_ordered_cooccurrence(df)
    
    # Run symmetric analysis  
    symmetric_matrix = analyze_symmetric_cooccurrence(df)
    
    # Optional: save results
    save_results = input("\nSave results to CSV? (y/n): ").lower() == 'y'
    if save_results:
        ordered_matrix.to_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index=True)
        symmetric_matrix.to_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index=True)
        print("Results saved to CSV files.")
    
    return ordered_matrix, symmetric_matrix

# Example usage:
# Assuming you have a DataFrame called 'df' with columns 'book_id' and 'genres'
# ordered_result, symmetric_result = main_analysis(df)
main_analysis(df)

In [7]:
ordered_matrix = pd.read_csv("ordered_genre_cooccurrence_unique_books_13x13.csv")
ordered_matrix

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,4,91,0,2,0,3,6,0,0,0
1,Adventure,0,0,9,46,35,6,17,1,27,14,1,6,23
2,Children's,0,284,0,137,93,66,30,4,56,11,13,0,0
3,Classics,1,17,5,0,286,11,10,6,23,25,7,1,0
4,Drama,53,16,1,83,0,1,21,0,26,18,8,0,2
5,Fantasy,2,812,99,14,98,0,35,193,236,16,232,17,40
6,Historical,3,31,9,14,286,4,0,1,38,44,59,1,7
7,Horror,0,6,8,15,33,42,0,0,194,3,23,6,97
8,Mystery,5,45,11,37,246,14,67,38,0,3,50,0,799
9,Nonfiction,18,62,6,88,585,7,35,1,19,0,10,41,6


In [8]:
symmetric_matrix = pd.read_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv")
symmetric_matrix

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,5,144,2,5,0,8,24,139,0,4
1,Adventure,0,0,293,63,51,818,48,7,72,76,19,264,78
2,Children's,0,293,0,142,94,165,39,12,67,17,22,12,0
3,Classics,5,63,142,0,369,25,24,21,60,113,44,35,0
4,Drama,144,51,94,369,0,99,307,33,272,603,941,65,28
5,Fantasy,2,818,165,25,99,0,39,235,250,23,351,41,40
6,Historical,5,48,39,24,307,39,0,1,105,79,189,8,13
7,Horror,0,7,12,21,33,235,1,0,232,4,55,46,123
8,Mystery,8,72,67,60,272,250,105,232,0,22,257,130,1088
9,Nonfiction,24,76,17,113,603,23,79,4,22,0,30,69,11


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# --- Your canonical 13 genres, fixed order for rows/cols ---
GENRES_13 = [
    "Adult", "Adventure", "Children's", "Classics", "Drama",
    "Fantasy", "Historical", "Horror", "Mystery", "Nonfiction",
    "Romance", "Science Fiction", "Thriller"
]

def parse_genres(s):
    if pd.isna(s):
        return []
    return [g.strip() for g in str(s).split(",") if g.strip()]

def to_canonical_set(gstr):
    glist = [g for g in parse_genres(gstr) if g in GENRES_13]
    # de-duplicate while preserving first-seen order
    return list(dict.fromkeys(glist))

def analyze_ordered_cooccurrence(df):
    """
    Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    """
    print("=== ORDERED CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books and keep one ordered genre list per book
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # Parse + filter to canonical 13, preserving order within each book
    books['genre_list'] = books['genres'].apply(parse_genres).apply(
        lambda gl: [g for g in gl if g in GENRES_13]
    )
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 2) Count and print how many UNIQUE canonical genres are actually present in df
    present = []
    for gl in books['genre_list']:
        present.extend(gl)
    present = sorted(set(present), key=lambda g: GENRES_13.index(g))
    
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 3) Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        # First occurrence positions per genre in this book (preserve order)
        pos = {}
        for k, g in enumerate(gl):
            if g not in pos:
                pos[g] = k
        
        # Count ordered pairs where Gi occurs BEFORE Gj (Gi != Gj). No diagonal.
        genres_in_book = list(pos.keys())
        for gi in genres_in_book:
            for gj in genres_in_book:
                if gi == gj:
                    continue
                if pos[gi] < pos[gj]:
                    co_mat[idx[gi], idx[gj]] += 1
    
    # Wrap into a 13x13 DataFrame with fixed order (diagonal is guaranteed 0)
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Ordered Co-occurrence Matrix:")
    print(co_df)
    return co_df

def analyze_symmetric_cooccurrence(df):
    """
    Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    """
    print("\n=== SYMMETRIC CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books; keep one genres string per book_id
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # 2) Parse & filter to canonical 13; also de-duplicate within a book
    books['genre_list'] = books['genres'].apply(to_canonical_set)
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 3) Report how many of the canonical 13 actually appear
    present = sorted(set(g for gl in books['genre_list'] for g in gl), key=lambda g: GENRES_13.index(g))
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 4) Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    #    Cell (Gi, Gj) = number of UNIQUE books that contain BOTH Gi and Gj (order-agnostic)
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        s = list(set(gl))           # ensure uniqueness before making pairs
        for g1, g2 in combinations(s, 2):
            i, j = idx[g1], idx[g2]
            # increment both directions to enforce symmetry
            co_mat[i, j] += 1
            co_mat[j, i] += 1
    
    # Zero the diagonal (no GiGi counts)
    np.fill_diagonal(co_mat, 0)
    
    # 5) Wrap in DataFrame
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Symmetric Co-occurrence Matrix:")
    print(co_df)
    return co_df

def main_analysis(df):
    """
    Run both analyses on the provided dataframe
    """
    print("Starting Genre Co-occurrence Analysis")
    print(f"Total rows in dataset: {len(df)}")
    
    # Run ordered analysis
    ordered_matrix = analyze_ordered_cooccurrence(df)
    
    # Run symmetric analysis  
    symmetric_matrix = analyze_symmetric_cooccurrence(df)
    
    # Optional: save results
    save_results = input("\nSave results to CSV? (y/n): ").lower() == 'y'
    if save_results:
        ordered_matrix.to_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index=True)
        symmetric_matrix.to_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index=True)
        print("Results saved to CSV files.")
    
    return ordered_matrix, symmetric_matrix

def analyze_genre_pairs(df):
    """
    Build a detailed pairwise analysis table and print formatted output
    """
    print("\n=== GENRE PAIR ANALYSIS ===")
    
    # 1) Unique books with ordered genre lists
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    books['genre_list'] = books['genres'].apply(parse_genres)
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 2) Discover all genres from the dataset
    GENRES = sorted({g for gl in books['genre_list'] for g in gl})
    print(f"Total unique genres found: {len(GENRES)}")
    
    # 3) Build a per-book position map for quick order comparisons
    pos_maps = []
    for gl in books['genre_list']:
        pos = {}
        for idx, g in enumerate(gl):
            if g not in pos:
                pos[g] = idx
        pos_maps.append(pos)
    
    # 4) Count for every pair
    def count_pair(g1, g2):
        total_both = 0
        g1_before_g2 = 0
        g2_before_g1 = 0
        for pos in pos_maps:
            if g1 in pos and g2 in pos:
                total_both += 1
                if pos[g1] < pos[g2]:
                    g1_before_g2 += 1
                elif pos[g2] < pos[g1]:
                    g2_before_g1 += 1
        return total_both, g1_before_g2, g2_before_g1
    
    # 5) Build tidy table and print formatted output
    rows = []
    print("\nDetailed pair analysis:")
    for i, g1 in enumerate(GENRES):
        for j in range(i+1, len(GENRES)):
            g2 = GENRES[j]
            total_both, g1_before_g2, g2_before_g1 = count_pair(g1, g2)
            
            # Print in requested format
            print(f"Total books in {g1}, {g2}: {total_both}")
            print(f"{g1.lower()}, {g2.lower()}: {g1_before_g2}")
            print(f"{g2.lower()}, {g1.lower()}: {g2_before_g1}")
            
            # Add to table
            rows.append({
                "Genre 1": g1,
                "Genre 2": g2,
                "Total Books (both)": total_both,
                "G1→G2 (g1 before g2)": g1_before_g2,
                "G2→G1 (g2 before g1)": g2_before_g1
            })
    
    # Create DataFrame
    pair_df = pd.DataFrame(rows).sort_values(["Genre 1", "Genre 2"]).reset_index(drop=True)
    
    # Add rates
    pair_df["G1→G2 rate"] = pair_df.apply(
        lambda r: (r["G1→G2 (g1 before g2)"] / r["Total Books (both)"]) if r["Total Books (both)"] else 0.0, axis=1
    )
    pair_df["G2→G1 rate"] = pair_df.apply(
        lambda r: (r["G2→G1 (g2 before g1)"] / r["Total Books (both)"]) if r["Total Books (both)"] else 0.0, axis=1
    )
    
    print("\nPair analysis table:")
    print(pair_df.to_string(index=False))
    
    # Build matrices
    all_genres = GENRES
    sym_totals = pd.DataFrame(0, index=all_genres, columns=all_genres, dtype=int)
    for _, r in pair_df.iterrows():
        g1, g2, t = r["Genre 1"], r["Genre 2"], int(r["Total Books (both)"])
        sym_totals.loc[g1, g2] = t
        sym_totals.loc[g2, g1] = t
    
    return pair_df, sym_totals

def main_analysis_extended(df):
    """
    Run all three analyses on the provided dataframe
    """
    print("Starting Extended Genre Co-occurrence Analysis")
    print(f"Total rows in dataset: {len(df)}")
    
    # Run ordered analysis
    ordered_matrix = analyze_ordered_cooccurrence(df)
    
    # Run symmetric analysis  
    symmetric_matrix = analyze_symmetric_cooccurrence(df)
    
    # Run pair analysis
    pair_df, pair_matrix = analyze_genre_pairs(df)
    
    # Optional: save results
    save_results = input("\nSave results to CSV? (y/n): ").lower() == 'y'
    if save_results:
        ordered_matrix.to_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index=True)
        symmetric_matrix.to_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index=True)
        pair_df.to_csv("genre_pair_order_table.csv", index=False)
        pair_matrix.to_csv("genre_pair_totals_symmetric.csv", index=True)
        print("Results saved to CSV files.")
    
    return ordered_matrix, symmetric_matrix, pair_df, pair_matrix

# Example usage:
# Assuming you have a DataFrame called 'df' with columns 'book_id' and 'genres'
# ordered_result, symmetric_result, pair_result, pair_matrix = main_analysis_extended(df)
main_analysis_extended(df)

In [13]:
pair_df =  pd.read_csv("genre_pair_order_table.csv")
pair_df

,Genre 1,Genre 2,Total Books (both),G1→G2 (g1 before g2),G2→G1 (g2 before g1),G1→G2 rate,G2→G1 rate
0,Adult,Adventure,0,0,0,0.000000,0.000000
1,Adult,Children's,0,0,0,0.000000,0.000000
2,Adult,Classics,5,4,1,0.800000,0.200000
3,Adult,Drama,144,91,53,0.631944,0.368056
4,Adult,Fantasy,2,0,2,0.000000,1.000000
...,...,...,...,...,...,...,...
73,Nonfiction,Science Fiction,69,41,28,0.594203,0.405797
74,Nonfiction,Thriller,11,6,5,0.545455,0.454545
75,Romance,Science Fiction,24,2,22,0.083333,0.916667
76,Romance,Thriller,60,58,2,0.966667,0.033333


In [14]:
pair_matrix = pd.read_csv("genre_pair_totals_symmetric.csv")
pair_matrix

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,5,144,2,5,0,8,24,139,0,4
1,Adventure,0,0,293,63,51,818,48,7,72,76,19,264,78
2,Children's,0,293,0,142,94,165,39,12,67,17,22,12,0
3,Classics,5,63,142,0,369,25,24,21,60,113,44,35,0
4,Drama,144,51,94,369,0,99,307,33,272,603,941,65,28
5,Fantasy,2,818,165,25,99,0,39,235,250,23,351,41,40
6,Historical,5,48,39,24,307,39,0,1,105,79,189,8,13
7,Horror,0,7,12,21,33,235,1,0,232,4,55,46,123
8,Mystery,8,72,67,60,272,250,105,232,0,22,257,130,1088
9,Nonfiction,24,76,17,113,603,23,79,4,22,0,30,69,11


In [12]:
ordered_matrix = pd.read_csv("ordered_genre_cooccurrence_unique_books_13x13.csv")
ordered_matrix 


,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,4,91,0,2,0,3,6,0,0,0
1,Adventure,0,0,9,46,35,6,17,1,27,14,1,6,23
2,Children's,0,284,0,137,93,66,30,4,56,11,13,0,0
3,Classics,1,17,5,0,286,11,10,6,23,25,7,1,0
4,Drama,53,16,1,83,0,1,21,0,26,18,8,0,2
5,Fantasy,2,812,99,14,98,0,35,193,236,16,232,17,40
6,Historical,3,31,9,14,286,4,0,1,38,44,59,1,7
7,Horror,0,6,8,15,33,42,0,0,194,3,23,6,97
8,Mystery,5,45,11,37,246,14,67,38,0,3,50,0,799
9,Nonfiction,18,62,6,88,585,7,35,1,19,0,10,41,6


In [ ]:
symmetric_matrix = pd.read_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv")
symmetric_matrix 

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,5,144,2,5,0,8,24,139,0,4
1,Adventure,0,0,293,63,51,818,48,7,72,76,19,264,78
2,Children's,0,293,0,142,94,165,39,12,67,17,22,12,0
3,Classics,5,63,142,0,369,25,24,21,60,113,44,35,0
4,Drama,144,51,94,369,0,99,307,33,272,603,941,65,28
5,Fantasy,2,818,165,25,99,0,39,235,250,23,351,41,40
6,Historical,5,48,39,24,307,39,0,1,105,79,189,8,13
7,Horror,0,7,12,21,33,235,1,0,232,4,55,46,123
8,Mystery,8,72,67,60,272,250,105,232,0,22,257,130,1088
9,Nonfiction,24,76,17,113,603,23,79,4,22,0,30,69,11


In [13]:
import re
import pandas as pd

# collapse to unique books
books_raw = (
    df[['book_id', 'original_title', 'authors', 'genres']]
      .dropna(subset=['book_id'])
      .sort_values('book_id')
      .drop_duplicates(subset=['book_id'], keep='first')
      .copy()
)

def split_genres(s):
    if pd.isna(s):
        return []
    return [g.strip() for g in str(s).split(",") if g.strip()]

tmp = books_raw.dropna(subset=['genres']).copy()
tmp['genre_list_raw'] = tmp['genres'].apply(split_genres)

def has_both(gl, a, b):
    s = set(gl)
    return a in s and b in s

pair = ("Adventure", "Fantasy")
mask_pair = tmp['genre_list_raw'].apply(lambda gl: has_both(gl, *pair))
subset = tmp[mask_pair]

print(f"Books with BOTH {pair[0]} and {pair[1]}:", len(subset))
display(subset[['book_id','original_title','authors','genres']].head(20))


Books with BOTH Adventure and Fantasy: 818


,book_id,original_title,authors,genres
3246005,6,The Fault in Our Stars,John Green,"Fantasy, Adventure"
1635898,19,The Fellowship of the Ring,J.R.R. Tolkien,"Fantasy, Adventure"
123194,21,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
3940906,23,Harry Potter and the Chamber of Secrets,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
2917592,24,Harry Potter and the Goblet of Fire,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
2740016,25,Harry Potter and the Deathly Hallows,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
3034951,27,Harry Potter and the Half-Blood Prince,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
3679511,36,The Giver,Lois Lowry,"Fantasy, Adventure"
576279,38,The Time Traveler's Wife,Audrey Niffenegger,"Fantasy, Adventure"
427492,41,The Lightning Thief,Rick Riordan,"Fantasy, Adventure"


In [25]:
import pandas as pd
import numpy as np

# Load the matrices
symmetric_matrix = pd.read_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index_col=0)
ordered_matrix = pd.read_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index_col=0)

# Count total possible genre pairs
total_genres = len(symmetric_matrix.columns)
total_possible_pairs = (total_genres * (total_genres - 1)) // 2  # C(n,2)
print(f"Total genres: {total_genres}")
print(f"Total possible genre pairs: {total_possible_pairs}")

# Count non-zero pairs in upper triangle only (to avoid double counting)
upper_triangle = np.triu(symmetric_matrix.values, k=1)  # Upper triangle excluding diagonal
non_zero_pairs = np.count_nonzero(upper_triangle)
zero_pairs = total_possible_pairs - non_zero_pairs

print(f"Non-zero genre pairs: {non_zero_pairs}")
print(f"Zero genre pairs: {zero_pairs}")

# Count zeros in full matrices
symmetric_zeros = (symmetric_matrix == 0).sum().sum()
ordered_zeros = (ordered_matrix == 0).sum().sum()
print(f"Total zeros in symmetric matrix: {symmetric_zeros}")
print(f"Total zeros in ordered matrix: {ordered_zeros}")

# Show pairs that never co-occur
print(f"\n=== PAIRS THAT NEVER CO-OCCUR ({zero_pairs} pairs) ===")
genres = ordered_matrix.index.tolist()
never_cooccur = []

for i in range(len(genres)):
    for j in range(i+1, len(genres)):
        g1, g2 = genres[i], genres[j]
        total = symmetric_matrix.loc[g1, g2]
        if total == 0:
            never_cooccur.append(f"{g1} - {g2}")
            print(f"{g1} - {g2}")

# Show ALL genre pairs with their directional counts
print(f"\n=== ALL GENRE PAIRS (78 total) ===")

for i in range(len(genres)):
    for j in range(i+1, len(genres)):
        g1, g2 = genres[i], genres[j]
        g1_to_g2 = ordered_matrix.loc[g1, g2]
        g2_to_g1 = ordered_matrix.loc[g2, g1]
        total = symmetric_matrix.loc[g1, g2]
        
        if total == 0:
            print(f"{g1}-{g2}: NEVER CO-OCCUR")
        else:
            print(f"{g1}-{g2}: {g1}→{g2}={g1_to_g2}, {g2}→{g1}={g2_to_g1}, Total={total}")

print(f"\n=== SUMMARY ===")
print(f"Pairs that never co-occur: {zero_pairs}")
print(f"Never co-occur list: {', '.join(never_cooccur)}")
print(f"Pairs that co-occur: {non_zero_pairs}")
print(f"Total pairs: {total_possible_pairs}")

Total genres: 13
Total possible genre pairs: 78
Non-zero genre pairs: 72
Zero genre pairs: 6
Total zeros in symmetric matrix: 25
Total zeros in ordered matrix: 34

=== PAIRS THAT NEVER CO-OCCUR (6 pairs) ===
Adult - Adventure
Adult - Children's
Adult - Horror
Adult - Science Fiction
Children's - Thriller
Classics - Thriller

=== ALL GENRE PAIRS (78 total) ===
Adult-Adventure: NEVER CO-OCCUR
Adult-Children's: NEVER CO-OCCUR
Adult-Classics: Adult→Classics=4, Classics→Adult=1, Total=5
Adult-Drama: Adult→Drama=91, Drama→Adult=53, Total=144
Adult-Fantasy: Adult→Fantasy=0, Fantasy→Adult=2, Total=2
Adult-Historical: Adult→Historical=2, Historical→Adult=3, Total=5
Adult-Horror: NEVER CO-OCCUR
Adult-Mystery: Adult→Mystery=3, Mystery→Adult=5, Total=8
Adult-Nonfiction: Adult→Nonfiction=6, Nonfiction→Adult=18, Total=24
Adult-Romance: Adult→Romance=0, Romance→Adult=139, Total=139
Adult-Science Fiction: NEVER CO-OCCUR
Adult-Thriller: Adult→Thriller=0, Thriller→Adult=4, Total=4
Adventure-Children's: 

In [5]:
# genre_pair_viz.py
# Parse your text block; produce:
# - pair_stats.csv
# - ordered_matrix_A_to_B.csv
# - symmetric_totals_matrix.csv
# - zero_pairs.csv
# - top20_pairs.csv
# And figures:
# - heatmap_symmetric_totals.png
# - heatmap_ordered_A_to_B.png
# - top20_pairs_bar.png
# - network_top_edges.png

import re
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RAW_TEXT = r"""<PASTE YOUR BLOCK HERE>"""

# ---- Parse lines into structured rows ----
pair_rows = []
for line in RAW_TEXT.splitlines():
    t = line.strip()
    if not t or t.startswith("==="):
        continue
    if ":" in t and ("→" in t or ("NEVER CO-OCCUR" in t and "-" in t)):
        m = re.match(r"^([A-Za-z' ]+)-([A-Za-z' ]+):\s*(.*)$", t)
        if not m: 
            continue
        a, b, rhs = m.groups()
        a, b = a.strip(), b.strip()
        if "NEVER CO-OCCUR" in rhs:
            pair_rows.append((a, b, 0, 0, 0, True))
        else:
            dir_matches = re.findall(r"([A-Za-z' ]+)→([A-Za-z' ]+)=([0-9]+)", rhs)
            total_m = re.search(r"Total=([0-9]+)", rhs)
            d = {(src.strip(), dst.strip()): int(v) for (src, dst, v) in dir_matches}
            total = int(total_m.group(1)) if total_m else None
            ab = d.get((a, b), 0)
            ba = d.get((b, a), 0)
            pair_rows.append((a, b, ab, ba, total if total is not None else ab + ba, False))

df_pairs = pd.DataFrame(pair_rows, columns=["A", "B", "A_to_B", "B_to_A", "Total", "Never"])
genres = sorted(set(df_pairs["A"]).union(df_pairs["B"]))

# Ordered matrix A->B
ordered = pd.DataFrame(0, index=genres, columns=genres, dtype=int)
for _, r in df_pairs.iterrows():
    ordered.loc[r["A"], r["B"]] = int(r["A_to_B"])
    ordered.loc[r["B"], r["A"]] = int(r["B_to_A"])

# Symmetric totals matrix
sym = pd.DataFrame(0, index=genres, columns=genres, dtype=int)
for _, r in df_pairs.iterrows():
    t = int(r["Total"])
    sym.loc[r["A"], r["B"]] = t
    sym.loc[r["B"], r["A"]] = t
np.fill_diagonal(sym.values, 0)

# Zero pairs
df_zero = df_pairs[df_pairs["Total"] == 0].assign(pair=lambda d: d["A"] + " - " + d["B"])[["pair"]]

# Top pairs
top_pairs = df_pairs.sort_values("Total", ascending=False).head(20).copy()
top_pairs["pair"] = top_pairs["A"] + " - " + top_pairs["B"]

outdir = Path("genre_pair_viz_outputs")
outdir.mkdir(exist_ok=True, parents=True)

# Save CSVs
df_pairs.to_csv(outdir / "pair_stats.csv", index=False)
ordered.to_csv(outdir / "ordered_matrix_A_to_B.csv")
sym.to_csv(outdir / "symmetric_totals_matrix.csv")
df_zero.to_csv(outdir / "zero_pairs.csv", index=False)
top_pairs[["pair", "Total"]].to_csv(outdir / "top20_pairs.csv", index=False)

# ---- Plot helpers (matplotlib only) ----
def plot_heatmap(mat: pd.DataFrame, title: str, outfile: Path):
    fig, ax = plt.subplots(figsize=(10, 8), dpi=160)
    im = ax.imshow(mat.values, aspect='auto')
    ax.set_xticks(range(len(mat.columns)))
    ax.set_yticks(range(len(mat.index)))
    ax.set_xticklabels(mat.columns, rotation=90)
    ax.set_yticklabels(mat.index)
    ax.set_title(title)
    fig.colorbar(im, ax=ax, shrink=0.8)
    fig.tight_layout()
    fig.savefig(outfile)
    plt.close(fig)

def plot_top_bar(df_sorted: pd.DataFrame, title: str, outfile: Path):
    fig, ax = plt.subplots(figsize=(12, 7), dpi=160)
    x = np.arange(len(df_sorted))
    vals = df_sorted["Total"].to_numpy()
    ax.bar(x, vals)
    ax.set_xticks(x)
    ax.set_xticklabels(df_sorted["pair"].tolist(), rotation=90)
    ax.set_ylabel("Co-occurrence (Total)")
    ax.set_title(title)
    for i, v in enumerate(vals):
        ax.text(i, v, str(v), ha='center', va='bottom', fontsize=8)
    fig.tight_layout()
    fig.savefig(outfile)
    plt.close(fig)

def plot_network(df_pairs: pd.DataFrame, title: str, outfile: Path, min_total: int = 100):
    import networkx as nx
    g = nx.Graph()
    g.add_nodes_from(genres)
    for _, r in df_pairs.iterrows():
        if r["Total"] >= min_total:
            g.add_edge(r["A"], r["B"], weight=r["Total"])
    pos = nx.spring_layout(g, seed=42)
    degs = dict(g.degree(weight="weight"))
    sizes = np.array([degs.get(n, 1) for n in g.nodes()]) * 10 + 100
    widths = [max(1.0, data["weight"]/100.0) for *_, data in g.edges(data=True)]
    fig, ax = plt.subplots(figsize=(9,7), dpi=160)
    nx.draw_networkx_nodes(g, pos, node_size=sizes, ax=ax)
    nx.draw_networkx_edges(g, pos, width=widths, ax=ax)
    nx.draw_networkx_labels(g, pos, font_size=9, ax=ax)
    ax.set_title(f"{title} (edges with Total ≥ {min_total})")
    ax.set_axis_off()
    fig.tight_layout()
    fig.savefig(outfile)
    plt.close(fig)

# ---- Create the figures ----
plot_heatmap(sym, "Genre Co-occurrence (Symmetric Totals)", outdir / "heatmap_symmetric_totals.png")
plot_heatmap(ordered, "Directed Co-occurrence (A → B)", outdir / "heatmap_ordered_A_to_B.png")
plot_top_bar(top_pairs.iloc[::-1], "Top 20 Genre Pairs by Total (ascending order for readability)", outdir / "top20_pairs_bar.png")
plot_network(df_pairs, "Genre Pair Network", outdir / "network_top_edges.png", min_total=100)

print(f"Done. Outputs saved under: {outdir.resolve()}")


/tmp/ipykernel_3675589/2960701599.py:82: UserWarning: Attempting to set identical left == right == -0.5 results in singular transformations; automatically expanding.
  im = ax.imshow(mat.values, aspect='auto')
/tmp/ipykernel_3675589/2960701599.py:82: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  im = ax.imshow(mat.values, aspect='auto')


Done. Outputs saved under: /home/moshtasa/Research/phd-svd-recsys/SVD/Book/notebook/1015/genre_pair_viz_outputs


In [ ]:
git add .
git commit -m "new exeriment"
git push

## adding synthetic

syntheyic users are availble at syn.py

In [7]:
import re
import pandas as pd
from pathlib import Path

PAIR_ROOT = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/1015/data/result/rec/top_re/1015/PAIR_INJECTION/5")
ORIGINAL_PATH = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv")

files = sorted(PAIR_ROOT.glob("fpair_*.csv"))
assert files, f"No fpair_*.csv files found in {PAIR_ROOT}"
fp = files[0]
print(f"🔎 Testing file: {fp.name}")

m = re.match(r"^fpair_(.+)__(.+)_(\d+)u_pos(\d+)_neg(\d+|NA)_(.+)\.csv$", fp.name)
assert m, f"Filename doesn't match pattern: {fp.name}"
g1, g2, run_users, pos_rating, neg, mode = m.groups()
run_users, pos_rating = int(run_users), int(pos_rating)

# Load original to find max original user id
orig_df = pd.read_csv(ORIGINAL_PATH, usecols=["user_id"])
orig_max_uid = int(orig_df["user_id"].max())

# Load poisoned file
df = pd.read_csv(fp, dtype={"user_id":"int64","book_id":"int64","rating":"float64","genres":"string"}, low_memory=False)

# Keep ONLY synthetic rows
synth = df[df["user_id"] > orig_max_uid].copy()
assert not synth.empty, "No synthetic users found."

def parse_genres(x):
    if x is None or (isinstance(x, float) and pd.isna(x)): return []
    return [g.strip() for g in str(x).split(",") if g.strip()]

# Identify pair vs non-pair on a sample
sample = synth.sample(min(20000, len(synth)), random_state=0).copy()
is_pair_book = sample["genres"].apply(lambda s: set(parse_genres(s)).issuperset({g1, g2}))

# Verify ratings
bad_pos = sample.loc[ is_pair_book & (sample["rating"] != float(pos_rating))]
bad_neg = sample.loc[~is_pair_book & (sample["rating"] != 0.0)]

if len(bad_pos) == 0 and len(bad_neg) == 0:
    print("✅ Strong poisoning verified on sample (pair=pos_rating, others=0).")
else:
    if len(bad_pos): print(f"❌ {len(bad_pos)} sampled pair-book rows not rated {pos_rating}")
    if len(bad_neg): print(f"❌ {len(bad_neg)} sampled non-pair rows not rated 0")

# ✅ Per-user completeness: compare to the synthetic book universe
attack_book_universe = synth["book_id"].nunique()
check_users = sample["user_id"].drop_duplicates().head(3)
counts = synth[synth["user_id"].isin(check_users)].groupby("user_id")["book_id"].nunique()

if (counts == attack_book_universe).all():
    print(f"✅ Each checked synthetic user has exactly one rating per ATTACK book (={attack_book_universe}).")
else:
    print("⚠️ Some synthetic users seem short vs. the attack universe:", counts.to_dict())


🔎 Testing file: fpair_Adult__Classics_100u_pos5_neg0_all.csv
✅ Strong poisoning verified on sample (pair=pos_rating, others=0).
✅ Each checked synthetic user has exactly one rating per ATTACK book (=9415).
